In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

import pyarrow as pa
import pyarrow.parquet as pq

import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


In [6]:
cwd = os.getcwd()

trade_data = pq.read_table(cwd + "\\data\\total_trade_data.parquet").to_pandas()

trade_data["time"] = pd.to_datetime(trade_data.time)

trade_data.set_index(["area_fips", "time"],inplace = True)

In [3]:
trade_data.head()

exposure = pd.qcut(trade_data.xs('2018-12-1', level=1).tariff, 4 ,labels = False)

most_exposed = exposure[exposure == 3].index.tolist()

trade_data.loc[most_exposed].xs('2018-12-1', level=1).tariff.mean()

6.515098948465189

In [3]:
empl_time_dict_16 = {"January Employment":dt.datetime(2016,1,1),
                 "February Employment":dt.datetime(2016,2,1),
                 "March Employment":dt.datetime(2016,3,1),
                 "April Employment":dt.datetime(2016,4,1),
                 "May Employment":dt.datetime(2016,5,1),
                 "June Employment":dt.datetime(2016,6,1),
                 "July Employment":dt.datetime(2016,7,1),
                 "August Employment":dt.datetime(2016,8,1),
                 "September Employment":dt.datetime(2016,9,1),
                 "October Employment":dt.datetime(2016,10,1),
                 "November Employment":dt.datetime(2016,11,1),
                 "December Employment":dt.datetime(2016,12,1),}

In [4]:
empl_time_dict_17 = {"January Employment":dt.datetime(2017,1,1),
                 "February Employment":dt.datetime(2017,2,1),
                 "March Employment":dt.datetime(2017,3,1),
                 "April Employment":dt.datetime(2017,4,1),
                 "May Employment":dt.datetime(2017,5,1),
                 "June Employment":dt.datetime(2017,6,1),
                 "July Employment":dt.datetime(2017,7,1),
                 "August Employment":dt.datetime(2017,8,1),
                 "September Employment":dt.datetime(2017,9,1),
                 "October Employment":dt.datetime(2017,10,1),
                 "November Employment":dt.datetime(2017,11,1),
                 "December Employment":dt.datetime(2017,12,1),}

empl_time_dict_18 = {"January Employment":dt.datetime(2018,1,1),
                 "February Employment":dt.datetime(2018,2,1),
                 "March Employment":dt.datetime(2018,3,1),
                 "April Employment":dt.datetime(2018,4,1),
                 "May Employment":dt.datetime(2018,5,1),
                 "June Employment":dt.datetime(2018,6,1),
                 "July Employment":dt.datetime(2018,7,1),
                 "August Employment":dt.datetime(2018,8,1),
                 "September Employment":dt.datetime(2018,9,1),
                 "October Employment":dt.datetime(2018,10,1),
                 "November Employment":dt.datetime(2018,11,1),
                 "December Employment":dt.datetime(2018,12,1),}

clistQ1 = ['Area\nCode','NAICS','Qtr','January Employment', 'February Employment',
       'March Employment', 'Total Quarterly Wages', 'Average Weekly Wage','Own',"Area Type"]

In [7]:
#url = "https://data.bls.gov/cew/data/files/2016/xls/2016_all_county_high_level.zip"
# This will read in the annual, single file. It's big, but has all we want...

#r = requests.get(url) 

# convert bytes to zip file  
#bls_q2016 = zf.ZipFile(io.BytesIO(r.content)) 
#bls_q2016.extractall()

url = "https://data.bls.gov/cew/data/files/2017/xls/2017_all_county_high_level.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_q2017 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2017.extractall(cwd + "\\bls_files")

url = "https://data.bls.gov/cew/data/files/2018/xls/2018_all_county_high_level.zip"

r = requests.get(url) 

bls_q2018 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2018.extractall(cwd + "\\bls_files")

In [6]:
bls_q2016.filelist

[<ZipInfo filename='allhlcn16.xlsx' compress_type=deflate external_attr=0x20 file_size=6593821 compress_size=6410634>,
 <ZipInfo filename='allhlcn161.xlsx' compress_type=deflate external_attr=0x20 file_size=6819533 compress_size=6681375>,
 <ZipInfo filename='allhlcn162.xlsx' compress_type=deflate external_attr=0x20 file_size=6828394 compress_size=6688576>,
 <ZipInfo filename='allhlcn163.xlsx' compress_type=deflate external_attr=0x20 file_size=6830643 compress_size=6691855>,
 <ZipInfo filename='allhlcn164.xlsx' compress_type=deflate external_attr=0x20 file_size=6823250 compress_size=6672198>]

In [10]:
#root_name = "allhlcn16"

#quarter = ["1","2","3","4"]

df = pd.DataFrame([])

#for item in quarter:
    
#    file_name = root_name + item + ".xlsx"
    
#    df = df.append(clean_bls_quarter(file_name,empl_time_dict_16))
    
############################################################################

root_name = cwd + "\\bls_files\\"

root_name = root_name + "allhlcn17"

quarter = ["1","2","3","4"]

for item in quarter:
    
    file_name = root_name + item + ".xlsx"
    
    df = df.append(clean_bls_quarter(file_name,empl_time_dict_17))
    
############################################################################  
root_name = cwd + "\\bls_files\\"

root_name = root_name + "allhlcn18"

quarter = ["1","2","3","4"]

for item in quarter:
    
    file_name = root_name + item + ".xlsx"
    
    df = df.append(clean_bls_quarter(file_name,empl_time_dict_18))

In [8]:
def clean_bls_quarter(excell_sheet, time_dict):

    df = pd.read_excel(excell_sheet, sheet_name = "US_St_Cn_MSA")

# Take only private

    df = df[df["Own"] == 5] 

# Take aggregate

    #df = df[df["NAICS"] == 101] # Take goods producing 
    
    df = df[df["NAICS"] == 10] # Take all employment in all sectors

# Take only counties 
    df = df[df["Area Type"] == "County"] 

    df.rename({"Area\nCode": "GEOFIPS"},axis = 1, inplace = True)

    df["GEOFIPS"] = df["GEOFIPS"].astype(int)

    df.set_index("GEOFIPS", inplace = True)

    df = df.reindex(trade_data.index.get_level_values(0).unique().astype(int).tolist())

    df = df.iloc[:,[13,14,15]].reset_index()

    df = df.melt("GEOFIPS")

    df.replace(time_dict,inplace = True)

    df.rename({"variable":"time", "value":"emply_month", "GEOFIPS": "area_fips"}, axis = 1, inplace = True)
    
    df["area_fips"] = df["area_fips"].astype(str)
    
    df.set_index(["area_fips", "time"], inplace = True)
    

    return df

In [11]:
df.head()

,,emply_month
area_fips,time,
10001,2017-01-01,0.0
10003,2017-01-01,249992.0
10005,2017-01-01,64182.0
1001,2017-01-01,8087.0
1003,2017-01-01,58781.0


In [12]:
df.sort_values(["area_fips", "time"], inplace = True)

In [13]:
trade_employ = trade_data.merge(df, left_index = True, right_index = True, how = "left")

In [14]:
trade_employ.head(50)

total_exp_pc  china_exp_pc    tariff  emplvl_2017  \
area_fips time                                                            
10001     2017-01-01    512.529185     53.382853  1.172229       2843.0   
          2017-02-01    492.604071     43.723710  1.172278       2843.0   
          2017-03-01    560.679984     37.347986  1.172365       2843.0   
          2017-04-01    499.471573     29.463467  1.172366       2843.0   
          2017-05-01    508.584288     30.009914  1.172366       2843.0   
          2017-06-01    499.350776     26.428918  1.172366       2843.0   
          2017-07-01    482.861116     28.199912  1.172365       2843.0   
          2017-08-01    500.243692     30.920545  1.172365       2843.0   
          2017-09-01    485.835551     40.123629  1.172365       2843.0   
          2017-10-01    559.828124     69.302723  1.172365       2843.0   
          2017-11-01    561.543549     68.431336  1.172365       2843.0   
          2017-12-01    545.184928     55.044490  1.172365       2843.0   
          2018-01-01    508.771439     44.837035  1.172287       2843.0   
          2018-02-01    507.639236     40.387392  1.172372       2843.0   
          2018-03-01    578.638456     39.261744  1.172364       2843.0   
          2018-04-01    550.940726     32.838192  1.618647       2843.0   
          2018-05-01    570.086178     30.267253  1.618648       2843.0   
          2018-06-01    550.233815     25.724372  1.618648       2843.0   
          2018-07-01    517.268058     22.709404  2.658051       2843.0   
          2018-08-01    538.203378     22.653503  2.658051       2843.0   
          2018-09-01    503.607799     18.887739  2.671780       2843.0   
          2018-10-01    555.839818     22.324031  2.931847       2843.0   
          2018-11-01    548.307689     22.488479  2.851468       2843.0   
          2018-12-01    518.410176     21.394456  2.851468       2843.0   
          2019-01-01    521.269401     27.129946  2.849199       2843.0   
          2019-02-01    498.847293     31.295510  2.848524       2843.0   
          2019-03-01    566.896475     33.364345  2.848460       2843.0   
          2019-04-01    534.509388     25.752972  2.848459       2843.0   
          2019-05-01    546.931214     31.870304  2.848458       2843.0   
          2019-06-01    520.400377     35.623348  2.848457       2843.0   
          2019-07-01    524.598708     38.045936  2.848461       2843.0   
10003     2017-01-01    450.852962     32.076890  0.209534       9072.0   
          2017-02-01    453.880300     33.515038  0.209546       9072.0   
          2017-03-01    519.587139     36.356246  0.209546       9072.0   
          2017-04-01    470.952064     34.879408  0.209546       9072.0   
          2017-05-01    490.418255     37.241834  0.209546       9072.0   
          2017-06-01    511.618468     37.443495  0.209546       9072.0   
          2017-07-01    470.826074     34.782890  0.209546       9072.0   
          2017-08-01    498.230617     36.805304  0.209546       9072.0   
          2017-09-01    488.690287     35.740364  0.209546       9072.0   
          2017-10-01    515.160403     37.749583  0.209546       9072.0   
          2017-11-01    504.720766     38.655271  0.209546       9072.0   
          2017-12-01    508.742468     42.461185  0.209546       9072.0   
          2018-01-01    485.144282     35.334606  0.209546       9072.0   
          2018-02-01    467.671278     32.357024  0.209546       9072.0   
          2018-03-01    549.277041     41.574998  0.209546       9072.0   
          2018-04-01    511.414408     37.511122  0.210252       9072.0   
          2018-05-01    529.151612     39.658968  0.210252       9072.0   
          2018-06-01    535.112232     41.488485  0.210252       9072.0   
          2018-07-01    498.537646     39.089010  0.202224       9072.0   

                       fips  total_employment  emply_month  
area_fips time                                              

In [15]:
file_path = os.getcwd() + "\\data\\trade_employment_all.parquet"

pq.write_table(pa.Table.from_pandas(trade_employ.reset_index()), file_path)

In [16]:
trade_employ.corr()

,total_exp_pc,china_exp_pc,tariff,emplvl_2017,total_employment,emply_month
total_exp_pc,1.000000,0.705856,0.288051,0.041760,-0.027994,-0.029026
china_exp_pc,0.705856,1.000000,0.168035,-0.008813,-0.050778,-0.054351
tariff,0.288051,0.168035,1.000000,-0.016043,-0.073930,-0.070506
emplvl_2017,0.041760,-0.008813,-0.016043,1.000000,0.896478,0.896412
total_employment,-0.027994,-0.050778,-0.073930,0.896478,1.000000,0.998860
emply_month,-0.029026,-0.054351,-0.070506,0.896412,0.998860,1.000000


In [17]:
trade_employ.head(25)

total_exp_pc  china_exp_pc    tariff  emplvl_2017  \
area_fips time                                                            
10001     2017-01-01    512.529185     53.382853  1.172229       2843.0   
          2017-02-01    492.604071     43.723710  1.172278       2843.0   
          2017-03-01    560.679984     37.347986  1.172365       2843.0   
          2017-04-01    499.471573     29.463467  1.172366       2843.0   
          2017-05-01    508.584288     30.009914  1.172366       2843.0   
          2017-06-01    499.350776     26.428918  1.172366       2843.0   
          2017-07-01    482.861116     28.199912  1.172365       2843.0   
          2017-08-01    500.243692     30.920545  1.172365       2843.0   
          2017-09-01    485.835551     40.123629  1.172365       2843.0   
          2017-10-01    559.828124     69.302723  1.172365       2843.0   
          2017-11-01    561.543549     68.431336  1.172365       2843.0   
          2017-12-01    545.184928     55.044490  1.172365       2843.0   
          2018-01-01    508.771439     44.837035  1.172287       2843.0   
          2018-02-01    507.639236     40.387392  1.172372       2843.0   
          2018-03-01    578.638456     39.261744  1.172364       2843.0   
          2018-04-01    550.940726     32.838192  1.618647       2843.0   
          2018-05-01    570.086178     30.267253  1.618648       2843.0   
          2018-06-01    550.233815     25.724372  1.618648       2843.0   
          2018-07-01    517.268058     22.709404  2.658051       2843.0   
          2018-08-01    538.203378     22.653503  2.658051       2843.0   
          2018-09-01    503.607799     18.887739  2.671780       2843.0   
          2018-10-01    555.839818     22.324031  2.931847       2843.0   
          2018-11-01    548.307689     22.488479  2.851468       2843.0   
          2018-12-01    518.410176     21.394456  2.851468       2843.0   
          2019-01-01    521.269401     27.129946  2.849199       2843.0   

                       fips  total_employment  emply_month  
area_fips time                                              
10001     2017-01-01  10001           29514.0          0.0  
          2017-02-01  10001           29514.0          0.0  
          2017-03-01  10001           29514.0          0.0  
          2017-04-01  10001           29514.0          0.0  
          2017-05-01  10001           29514.0          0.0  
          2017-06-01  10001           29514.0          0.0  
          2017-07-01  10001           29514.0          0.0  
          2017-08-01  10001           29514.0          0.0  
          2017-09-01  10001           29514.0          0.0  
          2017-10-01  10001           29514.0          0.0  
          2017-11-01  10001           29514.0          0.0  
          2017-12-01  10001           29514.0          0.0  
          2018-01-01  10001           29514.0          0.0  
          2018-02-01  10001           29514.0          0.0  
          2018-03-01  10001           29514.0          0.0  
          2018-04-01  10001           29514.0          0.0  
          2018-05-01  10001           29514.0          0.0  
          2018-06-01  10001           29514.0          0.0  
          2018-07-01  10001           29514.0          0.0  
          2018-08-01  10001           29514.0          0.0  
          2018-09-01  10001           29514.0          0.0  
          2018-10-01  10001           29514.0          0.0  
          2018-11-01  10001           29514.0          0.0  
          2018-12-01  10001           29514.0          0.0  
          2019-01-01  10001           29514.0          NaN